In [1]:
# * how to prevent disconnection on Google Colab: https://stackoverflow.com/questions/57113226/how-to-prevent-google-colab-from-disconnecting

# connect to account
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
# enable display
%load_ext google.colab.data_table 

In [3]:
# Chunk processing: GCP as an upper limit for CPU usage, therefore I chunk process the dataset by date. 
# The function bellow takes a start date and an end date and produce a table retreiving data between these two dates. 

# start:  2013-08-29T12:00:00
# end : 2015-08-31T23:00:00


from google.cloud import bigquery
client = bigquery.Client(project='sfbikeshare')


def createChunk(start_date, end_date, table_id):
  
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("start_date", "STRING", start_date),
            bigquery.ScalarQueryParameter("end_date", "STRING", end_date),
        ]
        ,
        destination = table_id
    )

    df = client.query('''
        WITH T AS
        (SELECT * FROM sfbikeshare.statusJoinStationHrGeom WHERE DTHR BETWEEN @start_date AND @end_date)
        SELECT T1.DTHR as time1, T1.station_id AS ID1, T2.station_id AS ID2, T2.DTHR as time2, 
              T1.bikes_available, T1.docks_available, 	T1.dock_count AS capacity, T1.lat AS lat1, 
              T1.long AS long1, T2.lat AS lat2, T2.long AS long2, T2.dock_count AS others_capacity 
        FROM T T1, T T2
        WHERE (T1.station_id <> T2.station_id)  AND (TIME_DIFF(TIME(T1.DTHR), TIME(T2.DTHR), HOUR) =0) AND ((date_diff(DATE(T1.DTHR), DATE(T2.DTHR), DAY) =0))
        ORDER BY time1, id1, id2
    ''', job_config = job_config).to_dataframe()


    print("Created table: " + table_id)

  

In [4]:
# loop through the entire date range with an incremental of n days per chunk, and save each chunk table to a bigQuery table. 


import datetime
start_date = datetime.datetime(2013, 8, 29, 0,0,0) 
delta = datetime.timedelta(days= 100)
end_date = start_date + delta


last_date = datetime.datetime(2015, 8, 31, 0, 0, 0)


while end_date <= last_date:
  
    table_id= "sfbikeshare.sfbikeshare." +  "status_g_" +  start_date.date().strftime('%Y%m%d') + "_" + end_date.date().strftime('%Y%m%d')  
    
    start_date_in = start_date.isoformat()
    end_date_in = end_date.isoformat()

    createChunk(start_date_in, end_date_in, table_id)

    start_date = end_date + datetime.timedelta(days= 100)
    end_date += delta

Created table: sfbikeshare.sfbikeshare.status_g_20130829_20131207
Created table: sfbikeshare.sfbikeshare.status_g_20140317_20140317
Created table: sfbikeshare.sfbikeshare.status_g_20140625_20140625
Created table: sfbikeshare.sfbikeshare.status_g_20141003_20141003
Created table: sfbikeshare.sfbikeshare.status_g_20150111_20150111
Created table: sfbikeshare.sfbikeshare.status_g_20150421_20150421
Created table: sfbikeshare.sfbikeshare.status_g_20150730_20150730


In [6]:
# Union Chunk Tables
from google.cloud import bigquery

client = bigquery.Client(project='sfbikeshare')

start_date = datetime.datetime(2013, 8, 29, 0, 0, 0).isoformat()
last_date = datetime.datetime(2015, 8, 31, 0, 0, 0).isoformat()


table_id = "sfbikeshare.sfbikeshare.master_table_test2"
table_to_insert = 'sfbikeshare.status_20130829_20131207'

job_config = bigquery.QueryJobConfig(
    query_parameters=[
            bigquery.ScalarQueryParameter("start_date", "STRING", start_date),
            bigquery.ScalarQueryParameter("last_date", "STRING", last_date),
    ]
    ,
    destination = table_id
)


df = client.query('''
    SELECT * FROM `sfbikeshare.sfbikeshare.status_g_*`
    WHERE _TABLE_SUFFIX BETWEEN @start_date AND @last_date
    
  ''', job_config = job_config).to_dataframe()

df.head()

,time1,ID1,ID2,time2,bikes_available,docks_available,capacity,lat1,long1,lat2,long2,others_capacity
0,2014-03-17,80,82,2014-03-17,7.916667,7.083333,15,37.352601,-121.905733,37.798541,-122.400862,15
1,2014-03-17,82,32,2014-03-17,5.983333,9.016667,15,37.798541,-122.400862,37.385956,-122.083678,11
2,2014-06-25,84,31,2014-06-25,8.000000,7.000000,15,37.342725,-121.895617,37.400443,-122.108338,15
3,2014-03-17,32,82,2014-03-17,6.000000,5.000000,11,37.385956,-122.083678,37.798541,-122.400862,15
4,2014-06-25,32,84,2014-06-25,4.000000,7.000000,11,37.385956,-122.083678,37.342725,-121.895617,15


In [7]:
df.shape

(9646800, 12)

In [8]:
df.head(20)

,time1,ID1,ID2,time2,bikes_available,docks_available,capacity,lat1,long1,lat2,long2,others_capacity
0,2014-03-17,80,82,2014-03-17,7.916667,7.083333,15,37.352601,-121.905733,37.798541,-122.400862,15
1,2014-03-17,82,32,2014-03-17,5.983333,9.016667,15,37.798541,-122.400862,37.385956,-122.083678,11
2,2014-06-25,84,31,2014-06-25,8.000000,7.000000,15,37.342725,-121.895617,37.400443,-122.108338,15
3,2014-03-17,32,82,2014-03-17,6.000000,5.000000,11,37.385956,-122.083678,37.798541,-122.400862,15
4,2014-06-25,32,84,2014-06-25,4.000000,7.000000,11,37.385956,-122.083678,37.342725,-121.895617,15
5,2014-03-17,80,83,2014-03-17,7.916667,7.083333,15,37.352601,-121.905733,37.491269,-122.236234,15
6,2014-06-25,80,83,2014-06-25,7.916667,7.083333,15,37.352601,-121.905733,37.491269,-122.236234,15
7,2014-06-25,82,80,2014-06-25,5.000000,10.000000,15,37.798541,-122.400862,37.352601,-121.905733,15
8,2014-06-25,31,32,2014-06-25,8.000000,7.000000,15,37.400443,-122.108338,37.385956,-122.083678,11
9,2014-10-03,84,83,2014-10-03,7.000000,8.000000,15,37.342725,-121.895617,37.491269,-122.236234,15
